This notebook generates a map to answer answer the following question:
* What facilities (by industry) are upstream of sample points with specific types of PFAS contamination within some range?

# Setup


In [ ]:
%%capture
!pip install mapclassify
!pip install SPARQLWrapper
!pip install rdflib
!pip install ipython-autotime
%load_ext autotime

time: 341 µs (started: 2025-11-10 16:18:16 +00:00)


In [ ]:
from branca.element import Figure                                  # For controlling the size of the final map
import folium                                                      # For map layer control
import math                                                        # For math functions (specifically, math.log)
import geopandas as gpd                                            # For geospatial dataframes
import pandas as pd                                                # For dataframes
from shapely import wkt                                            # For working with WKT coordinates in a GeoDataFrame
from SPARQLWrapper import SPARQLWrapper2, JSON, GET, POST, DIGEST  # For querying SPARQL endpoints
import rdflib                                                      # For working with URIs

def convertToDataframe(results):
  d = []
  for x in results.bindings:
        row = {}
        for k in x:
            v = x[k]
            vv = rdflib.term.Literal(v.value, datatype=v.datatype).toPython()  # type: ignore[no-untyped-call]
            row[k] = vv
        d.append(row)
  df = pd.DataFrame(d)
  return df

def convertS2ListToQueryString(s2list):
  s2list_short = [s2cell.replace("http://stko-kwg.geog.ucsb.edu/lod/resource/","kwgr:") for s2cell in s2list]
  s2_values_string = " ".join(s2list_short)
  return s2_values_string


time: 1.41 s (started: 2025-11-10 16:18:16 +00:00)


In [ ]:
#for interactive widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

time: 82.2 ms (started: 2025-11-10 16:18:17 +00:00)


In [ ]:
frink_fed_endpoint = 'https://frink.apps.renci.org/federation/sparql'

sparqlGET = SPARQLWrapper2(frink_fed_endpoint)
sparqlGET.setHTTPAuth(DIGEST)
sparqlGET.setMethod(POST)
sparqlGET.setReturnFormat(JSON)

time: 1.07 ms (started: 2025-11-10 16:18:17 +00:00)


# Q1 - What facilities are upstream from specific types of samples with high PFAS concentrations?


In [ ]:
#Parameters to choose

substance = "PFOA" # @param ["PFOS", "PFOA", "PFBA", "PFBEA", "PFBS", "PFHPA", "PFHXS", "PFHXA", "PFHPS", "PFNA", "PFDA"]{"allow-input":true}
substanceCode = "me_egad:parameter." + substance + "_A"

materialType = "GW (Groundwater)" # @param ["DW (Drinking Water)", "GW (Groundwater)", "WW (Waste Water)", "SW (Surface Water)", "PW (Pore Water)", "L (Leachate)", "SR (Storm Water Runoff)", "SL (Soil)" ]{"allow-input":true}
matTypeCode = "me_egad_data:sampleMaterialType." + materialType.split()[0]

admin_region = "23005 (Cumberland County, Maine)" # @param ["23 (Maine)","23019 (Penobscot County, Maine)", "23011 (Kennebec County, Maine)", "23005 (Cumberland County, Maine)", "33 (New Hampshire)","17 (Illinois)"] {"allow-input":true}
regionCode = admin_region.split()[0]

minValue = 10 # @param
maxValue = 1000 # @param

#industry = "562 (Waste Treatment and Disposal)" # @param ["562 (Waste Treatment and Disposal)","322 (Paper Manufacturing)", "326 (Plastics and Rubber Products Manufacturing)","313 (Textile Mills)","321 (Wood Product Manufacturing)","325 (Chemical Manufacturing)", "336 (Aerospace Product and Part Manufacturing)"]{"allow-input":true}
#industryCode = industry.split()[0]



time: 4.11 ms (started: 2025-11-10 16:26:17 +00:00)


## Queries

In [ ]:
# Query sample points, sample data, and S2 cells within a given administrative region

query = '''
PREFIX coso: <http://w3id.org/coso/v1/contaminoso#>
PREFIX fio: <http://w3id.org/fio/v1/fio#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
PREFIX me_egad: <http://sawgraph.spatialai.org/v1/me-egad#>
PREFIX me_egad_data: <http://sawgraph.spatialai.org/v1/me-egad-data#>
PREFIX naics: <http://w3id.org/fio/v1/naics#>
PREFIX nhdplusv2: <http://nhdplusv2.spatialai.org/v1/nhdplusv2#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX spatial: <http://purl.org/spatialai/spatial/spatial-full#>

SELECT DISTINCT (COUNT(DISTINCT ?subVal) as ?resultCount) (MAX(?result_value) as ?max) ?sp ?spWKT ?upstream_flowlineWKT ?facility ?facWKT ?facilityName ?industryName ?industryGroup ?industryGroupName ?industrySubsector ?industrySubsectorName WHERE {
    ?sp rdf:type coso:SamplePoint ;
        geo:hasGeometry/geo:asWKT ?spWKT ;
        spatial:connectedTo ?ar3 ;
        spatial:connectedTo ?s2 .
    ?ar3 rdf:type kwg-ont:AdministrativeRegion_3 ;
         kwg-ont:administrativePartOf kwgr:administrativeRegion.USA.''' + regionCode + ''' .
    ?s2 rdf:type kwg-ont:S2Cell_Level13 .
    ?s2cell rdf:type kwg-ont:S2Cell_Level13 ;
             kwg-ont:sfTouches | owl:sameAs ?s2 .
    ?observation rdf:type coso:ContaminantObservation ;
                coso:observedAtSamplePoint ?sp ;
                coso:ofSubstance ?substance ;
                coso:analyzedSample ?sample ;
                coso:hasResult ?result .
    ?sample rdfs:label ?sampleLabel ;
            coso:sampleOfMaterialType ?matType .
    ?matType rdfs:label ?matTypeLabel .
    ?result coso:measurementValue ?result_value ;
            coso:measurementUnit ?unit .
    ?unit qudt:symbol ?unit_sym .
    VALUES ?unit {<http://qudt.org/vocab/unit/NanoGM-PER-L>}
    VALUES ?substance {''' + substanceCode + '''}
    VALUES ?matType {''' + matTypeCode + '''}
    FILTER (?result_value > '''+ str(minValue) + ''')
    FILTER (?result_value <  '''+ str(maxValue) + ''')
    BIND((CONCAT(str(?result_value) , " ", ?unit_sym)) as ?subVal)

    ?downstream_flowline rdf:type hyf:HY_FlowPath ;
						 spatial:connectedTo ?s2cell .
	# find all flowlines upstream of them
	?upstream_flowline hyf:downstreamFlowPathTC ?downstream_flowline ;
					   geo:hasGeometry/geo:asWKT ?upstream_flowlineWKT .
	?s2cellus spatial:connectedTo ?upstream_flowline ;
			  rdf:type kwg-ont:S2Cell_Level13 .

    #find facilities
    ?s2cellus kwg-ont:sfContains ?facility.
    ?facility fio:ofIndustry ?industryCode, ?industryGroup, ?industrySubsector ;
              geo:hasGeometry/geo:asWKT ?facWKT;
              rdfs:label ?facilityName.
    ?industryCode a naics:NAICS-IndustryCode;
                  rdfs:label ?industryName ;
    fio:subcodeOf ?industryGroup .
    ?industryGroup a naics:NAICS-IndustryGroup;
                   rdfs:label ?industryGroupName ;
                   fio:subcodeOf ?industrySubsector .
    ?industrySubsector a naics:NAICS-IndustrySubsector;
                       rdfs:label ?industrySubsectorName;
                       fio:subcodeOf naics:NAICS-31 .

} GROUP BY ?sp ?spWKT ?upstream_flowlineWKT ?facility ?facWKT ?facilityName ?industryName ?industryGroup ?industryGroupName ?industrySubsector ?industrySubsectorName
'''

sparqlGET.setQuery(query)
results = sparqlGET.query()
results_df = convertToDataframe(results)
# print(results_df.columns)

time: 12.1 s (started: 2025-11-10 16:26:17 +00:00)


In [ ]:
# # Use this query to pull all upstream flowlines
# # The first query only pulls the upstream flowlines near a facility
# # This query takes ~30 seconds

# query_hydro = '''
# PREFIX coso: <http://w3id.org/coso/v1/contaminoso#>
# PREFIX fio: <http://w3id.org/fio/v1/fio#>
# PREFIX geo: <http://www.opengis.net/ont/geosparql#>
# PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
# PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
# PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
# PREFIX me_egad: <http://sawgraph.spatialai.org/v1/me-egad#>
# PREFIX me_egad_data: <http://sawgraph.spatialai.org/v1/me-egad-data#>
# PREFIX naics: <http://w3id.org/fio/v1/naics#>
# PREFIX nhdplusv2: <http://nhdplusv2.spatialai.org/v1/nhdplusv2#>
# PREFIX owl: <http://www.w3.org/2002/07/owl#>
# PREFIX qudt: <http://qudt.org/schema/qudt/>
# PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
# PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
# PREFIX spatial: <http://purl.org/spatialai/spatial/spatial-full#>

# SELECT ?upstream_flowline ?upstream_flowlineWKT WHERE {
#     ?sp rdf:type coso:SamplePoint ;
#         spatial:connectedTo ?ar3 ;
#         spatial:connectedTo ?s2 .
#     ?ar3 rdf:type kwg-ont:AdministrativeRegion_3 ;
#          kwg-ont:administrativePartOf kwgr:administrativeRegion.USA.''' + regionCode + ''' .
#     ?s2 rdf:type kwg-ont:S2Cell_Level13 .
#     ?s2cell rdf:type kwg-ont:S2Cell_Level13 ;
#              kwg-ont:sfTouches | owl:sameAs ?s2 .
#     ?observation rdf:type coso:ContaminantObservation ;
#                 coso:observedAtSamplePoint ?sp ;
#                 coso:ofSubstance ?substance ;
#                 coso:analyzedSample ?sample ;
#                 coso:hasResult ?result .
#     ?sample rdfs:label ?sampleLabel ;
#             coso:sampleOfMaterialType ?matType .
#     ?matType rdfs:label ?matTypeLabel .
#     ?result coso:measurementValue ?result_value ;
#             coso:measurementUnit ?unit .
#     ?unit qudt:symbol ?unit_sym .
#     VALUES ?unit {<http://qudt.org/vocab/unit/NanoGM-PER-L>}
#     VALUES ?substance {''' + substanceCode + '''}
#     VALUES ?matType {''' + matTypeCode + '''}
#     FILTER (?result_value > '''+ str(minValue) + ''')
#     FILTER (?result_value <  '''+ str(maxValue) + ''')
#     BIND((CONCAT(str(?result_value) , " ", ?unit_sym)) as ?subVal)

#     ?downstream_flowline rdf:type hyf:HY_FlowPath ;
# 						 spatial:connectedTo ?s2cell .
# 	?upstream_flowline hyf:downstreamFlowPathTC ?downstream_flowline ;
# 					   geo:hasGeometry/geo:asWKT ?upstream_flowlineWKT .
# }
# '''

# sparqlGET.setQuery(query_hydro)
# hydrology_results = sparqlGET.query()
# hydrology_df = convertToDataframe(hydrology_results)
# # print(results_df.columns)

time: 2.14 ms (started: 2025-11-10 16:26:29 +00:00)


In [ ]:
# County boundaries for mapping
county_query = '''
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT * WHERE {
    ?county geo:hasGeometry/geo:asWKT ?countyWKT ;
            rdfs:label ?countyName.
    VALUES ?county {kwgr:administrativeRegion.USA.'''+ regionCode + '''}
}
'''

sparqlGET.setQuery(county_query)
county_results = sparqlGET.query()
county_df = convertToDataframe(county_results)

time: 516 ms (started: 2025-11-10 16:26:29 +00:00)


## Prep data for mapping

In [ ]:
samplepoints_columns = [ 'sp', 'resultCount', 'max', 'spWKT' ]
samplepoints_df = results_df[samplepoints_columns].copy()
samplepoints_df.drop_duplicates(inplace=True)
samplepoints_df['spWKT'] = samplepoints_df['spWKT'].apply(wkt.loads)
samplepoints_df = gpd.GeoDataFrame(samplepoints_df, geometry='spWKT')
samplepoints_df.set_crs(epsg=4326, inplace=True, allow_override=True)

,sp,resultCount,max,spWKT
0,http://sawgraph.spatialai.org/v1/me-egad-data#...,1,10.1,POINT (-70.45186 43.72595)
19,http://sawgraph.spatialai.org/v1/me-egad-data#...,1,10.3,POINT (-70.37741 43.64785)
174,http://sawgraph.spatialai.org/v1/me-egad-data#...,1,10.77,POINT (-69.91384 43.90278)
854,http://sawgraph.spatialai.org/v1/me-egad-data#...,1,10.8,POINT (-70.63581 44.14963)
857,http://sawgraph.spatialai.org/v1/me-egad-data#...,1,11.0,POINT (-69.93122 43.89672)
...,...,...,...,...
13011,http://sawgraph.spatialai.org/v1/me-egad-data#...,4,36.4,POINT (-70.379 43.64562)
13166,http://sawgraph.spatialai.org/v1/me-egad-data#...,4,970.0,POINT (-69.92734 43.88894)
13172,http://sawgraph.spatialai.org/v1/me-egad-data#...,5,808.0,POINT (-70.45057 43.72784)
13191,http://sawgraph.spatialai.org/v1/me-egad-data#...,7,818.0,POINT (-70.45376 43.72717)


time: 43.8 ms (started: 2025-11-10 16:26:30 +00:00)


In [ ]:
# # Use this block instead of the following one if only interested in some (not all) upstream flowlines

hydrology_columns = [ 'upstream_flowlineWKT' ]
hydrology_df = results_df[hydrology_columns].copy()
hydrology_df.drop_duplicates(inplace=True)
hydrology_df['upstream_flowlineWKT'] = hydrology_df['upstream_flowlineWKT'].apply(wkt.loads)
hydrology_df = gpd.GeoDataFrame(hydrology_df, geometry='upstream_flowlineWKT')
hydrology_df.set_crs(epsg=4326, inplace=True, allow_override=True)

,upstream_flowlineWKT
0,"LINESTRING (-70.44186 43.73233, -70.44132 43.7..."
2,"LINESTRING (-70.49773 43.69965, -70.49773 43.6..."
3,"LINESTRING (-70.53272 43.74955, -70.53222 43.7..."
11,"LINESTRING (-70.53768 43.75514, -70.53693 43.7..."
19,"LINESTRING (-70.35091 43.65527, -70.35053 43.6..."
...,...
3115,"LINESTRING (-70.14726 43.8231, -70.14726 43.82..."
3116,"LINESTRING (-70.15196 43.81446, -70.1522 43.81..."
3117,"LINESTRING (-70.15326 43.81295, -70.15265 43.8..."
3122,"LINESTRING (-70.15357 43.81308, -70.15352 43.8..."


time: 61.7 ms (started: 2025-11-10 16:26:30 +00:00)


In [ ]:
# # Use this block instead of the preceding one if attempting to map all upstream flowlines

# hydrology_df.drop_duplicates(inplace=True)
# hydrology_df['upstream_flowlineWKT'] = hydrology_df['upstream_flowlineWKT'].apply(wkt.loads)
# hydrology_df = gpd.GeoDataFrame(hydrology_df, geometry='upstream_flowlineWKT')
# hydrology_df.set_crs(epsg=4326, inplace=True, allow_override=True)

time: 442 µs (started: 2025-11-10 16:26:30 +00:00)


In [ ]:
facilities_columns = [ 'facility', 'facWKT', 'facilityName', 'industryName', 'industryGroup', 'industryGroupName', 'industrySubsector', 'industrySubsectorName' ]
facilities_df = results_df[facilities_columns].copy()
facilities_df.drop_duplicates(inplace=True)
facilities_df['facWKT'] = facilities_df['facWKT'].apply(wkt.loads)
facilities_df = gpd.GeoDataFrame(facilities_df, geometry='facWKT')
facilities_df.set_crs(epsg=4326, inplace=True, allow_override=True)

,facility,facWKT,facilityName,industryName,industryGroup,industryGroupName,industrySubsector,industrySubsectorName
0,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-70.4227 43.7266),RICH TOOL & DIE CO,Aerospace Product and Parts Manufacturing,http://w3id.org/fio/v1/naics#NAICS-3364,Aerospace Product and Parts Manufacturing,http://w3id.org/fio/v1/naics#NAICS-336,Transportation Equipment Manufacturing
1,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-70.4227 43.7266),RICH TOOL & DIE CO,Aircraft Engine and Engine Parts Manufacturing,http://w3id.org/fio/v1/naics#NAICS-3364,Aerospace Product and Parts Manufacturing,http://w3id.org/fio/v1/naics#NAICS-336,Transportation Equipment Manufacturing
2,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-70.5012 43.70196),ALLSTATE ENVIRONMENTAL SERVICES,Printing,http://w3id.org/fio/v1/naics#NAICS-3231,Printing and Related Support Activities,http://w3id.org/fio/v1/naics#NAICS-323,Printing and Related Support Activities
3,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-70.53833 43.74861),FIRST TECHNOLOGY/CONTROL DEVICES,All Other Electrical Equipment and Component M...,http://w3id.org/fio/v1/naics#NAICS-3359,Other Electrical Equipment and Component Manuf...,http://w3id.org/fio/v1/naics#NAICS-335,"Electrical Equipment, Appliance, and Component..."
4,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-70.53833 43.74861),FIRST TECHNOLOGY/CONTROL DEVICES,All Other Miscellaneous Electrical Equipment a...,http://w3id.org/fio/v1/naics#NAICS-3359,Other Electrical Equipment and Component Manuf...,http://w3id.org/fio/v1/naics#NAICS-335,"Electrical Equipment, Appliance, and Component..."
...,...,...,...,...,...,...,...,...
3115,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-70.1477 43.81584),DOWNEAST WOODWORKS,Wood Kitchen Cabinet and Countertop Manufacturing,http://w3id.org/fio/v1/naics#NAICS-3371,Household and Institutional Furniture and Kitc...,http://w3id.org/fio/v1/naics#NAICS-337,Furniture and Related Product Manufacturing
3117,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-70.16376 43.7998),ROYAL RIVER BOAT YARD,Boat Building,http://w3id.org/fio/v1/naics#NAICS-3366,Ship and Boat Building,http://w3id.org/fio/v1/naics#NAICS-336,Transportation Equipment Manufacturing
3118,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-70.16376 43.7998),ROYAL RIVER BOAT YARD,Ship and Boat Building,http://w3id.org/fio/v1/naics#NAICS-3366,Ship and Boat Building,http://w3id.org/fio/v1/naics#NAICS-336,Transportation Equipment Manufacturing
3120,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-70.15363 43.80716),"GREENE MARINE, INC.",Boat Building,http://w3id.org/fio/v1/naics#NAICS-3366,Ship and Boat Building,http://w3id.org/fio/v1/naics#NAICS-336,Transportation Equipment Manufacturing


time: 72.3 ms (started: 2025-11-10 16:26:30 +00:00)


In [ ]:
county_df['countyWKT'] = county_df['countyWKT'].apply(wkt.loads)
county_df = gpd.GeoDataFrame(county_df, geometry='countyWKT')
county_df.set_crs(epsg=4326, inplace=True, allow_override=True)

,county,countyWKT,countyName
0,http://stko-kwg.geog.ucsb.edu/lod/resource/adm...,"POLYGON ((-70.01893 43.58024, -70.02157 43.579...","Cumberland County, Maine"


time: 29.6 ms (started: 2025-11-10 16:26:30 +00:00)


In [ ]:
%%capture
map = county_df.explore(name='Counties',
                        style_kwds=dict(color='Gray',
                                        fill=0.0,
                                        weight=7))

samplepoints_df.explore(m=map,
                        name=f'<span style="color:DarkOrange;">Samples</span>',
                        color='DarkOrange',
                        style_kwds=dict(style_function=lambda x: { 'radius': math.log(float(x['properties']['max'])) * 1.8976 + 3.36937,  # fits to (4 ppt, 6 radius) and (6400 ppt, 20 radius)
                                                                   'opacity': 0.3,
                                                                   'color': 'DimGray',
                                                                 }
                                       ),
                        marker_kwds=dict(radius=6),
                        marker_type='circle_marker',
                        popup = [ 'sp', 'resultCount', 'max' ],
                        show=False)

hydrology_df.explore(m=map,
                     name='<span style="color:Blue;">Flowlines</span>',
                     color='Blue',
                     style_kwds=dict(weight=.5),
                     show=False)

c = 0
colors = ['MidnightBlue','MediumBlue','SlateBlue','MediumSlateBlue', 'DodgerBlue','DeepSkyBlue','SkyBlue','CadetBlue','DarkCyan','LightSeaGreen',
          'MediumSageGreen','PaleVioletRed','Purple','Orchid','Fuchsia','MediumVioletRed','HotPink','LightPink', 'lightblue', 'gray', 'blue', 'darkred',
          'lightgreen', 'purple', 'red', 'green', 'lightred', 'white', 'darkblue', 'darkpurple', 'cadetblue', 'orange', 'pink', 'lightgray', 'darkgreen']
for industry in list(facilities_df.industrySubsectorName.unique()):
    facilities_df[facilities_df['industrySubsectorName']==industry].explore(m=map,
                                                                            name=f'<span style="color:{colors[c]};">{industry}</span>',
                                                                            color=colors[c],
                                                                            marker_kwds=dict(radius=6,
                                                                                             fill=True),
                                                                            style_kwds=dict(fillOpacity=1),
                                                                            marker_type='circle_marker',
                                                                            popup=True,
                                                                            show=False)
    c += 1

time: 329 ms (started: 2025-11-10 16:26:30 +00:00)


## Map

In [ ]:
folium.LayerControl(collapsed=False).add_to(map)
fig = Figure(width='100%', height=700)
fig.add_child(map)

time: 315 ms (started: 2025-11-10 16:26:31 +00:00)


In [ ]:
fig.save('SAWGraph-demo_2025-09-28_TracingUpstream_Kennebec.html')

time: 382 ms (started: 2025-11-10 16:26:31 +00:00)
